In [21]:
import pandas as pd
import pyodbc
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateutil.rrule import rrule, MONTHLY
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from scipy.stats import halfnorm
import shap
import numdifftools as nd
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.pyll.base import scope
import pickle
shap.initjs()
%load_ext autoreload
%autoreload 2
import mmm_transformations
import mmm_preprocessing
import mmm_modeling
import mmm_response_curves
import mmm_optimization

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preprocessing

In [22]:
df_pp = pd.read_csv('sim_data_r (1).csv')
df_pp.drop(['Unnamed: 0'], axis=1, inplace=True)
df_pp = pd.concat([df_pp, pd.get_dummies(df_pp[['Specialty']])], axis=1)
df_pp

,Specialty,date,rx_count,Email,Phone,Digital,Specialty_Hematologist,Specialty_Neurologist,Specialty_Oncologist
0,Neurologist,1,199,185,523,44,0,1,0
1,Neurologist,2,167,3,910,446,0,1,0
2,Neurologist,3,212,81,864,147,0,1,0
3,Neurologist,4,214,141,998,131,0,1,0
4,Neurologist,5,214,38,888,427,0,1,0
...,...,...,...,...,...,...,...,...,...
355,Hematologist,116,219,30,590,234,1,0,0
356,Hematologist,117,206,20,287,422,1,0,0
357,Hematologist,118,99,182,934,2,1,0,0
358,Hematologist,119,251,174,808,410,1,0,0


# Transformations

In [23]:
transform = mmm_transformations.MMMTransformations()

In [24]:
df_t = transform.lag_dv(df_pp, 'rx_count', 3, 'Specialty')
df_t = transform.lag_dv(df_t, 'Email', 3, 'Specialty')
df_t = transform.lag_dv(df_t, 'Phone', 3, 'Specialty')
df_t = transform.lag_dv(df_t, 'Digital', 3, 'Specialty')
df_t

,Specialty,date,rx_count,Email,Phone,Digital,Specialty_Hematologist,Specialty_Neurologist,Specialty_Oncologist,rx_count_lag1,...,rx_count_lag3,Email_lag1,Email_lag2,Email_lag3,Phone_lag1,Phone_lag2,Phone_lag3,Digital_lag1,Digital_lag2,Digital_lag3
0,Neurologist,1,199,185,523,44,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Neurologist,2,167,3,910,446,0,1,0,199,...,0,185,0,0,523,0,0,44,0,0
2,Neurologist,3,212,81,864,147,0,1,0,167,...,0,3,185,0,910,523,0,446,44,0
3,Neurologist,4,214,141,998,131,0,1,0,212,...,199,81,3,185,864,910,523,147,446,44
4,Neurologist,5,214,38,888,427,0,1,0,214,...,167,141,81,3,998,864,910,131,147,446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Hematologist,116,219,30,590,234,1,0,0,212,...,245,9,71,102,741,239,577,438,59,274
356,Hematologist,117,206,20,287,422,1,0,0,219,...,202,30,9,71,590,741,239,234,438,59
357,Hematologist,118,99,182,934,2,1,0,0,206,...,212,20,30,9,287,590,741,422,234,438
358,Hematologist,119,251,174,808,410,1,0,0,99,...,219,182,20,30,934,287,590,2,422,234


In [25]:
df_t.describe()

,date,rx_count,Email,Phone,Digital,Specialty_Hematologist,Specialty_Neurologist,Specialty_Oncologist,rx_count_lag1,rx_count_lag2,rx_count_lag3,Email_lag1,Email_lag2,Email_lag3,Phone_lag1,Phone_lag2,Phone_lag3,Digital_lag1,Digital_lag2,Digital_lag3
count,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000
mean,60.500000,184.566667,95.791667,525.305556,256.455556,0.333333,0.333333,0.333333,183.069444,181.369444,180.277778,95.316667,94.300000,93.647222,521.122222,514.875000,510.358333,254.358333,252.591667,250.600000
std,34.688025,41.707444,58.012999,284.562531,139.537490,0.472061,0.472061,0.472061,44.798380,47.615740,50.152476,58.493147,58.641577,58.839288,287.036877,290.002898,292.253147,141.135040,142.322567,143.267212
min,1.000000,50.000000,0.000000,7.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.750000,155.750000,47.000000,293.500000,139.750000,0.000000,0.000000,0.000000,155.000000,154.750000,154.750000,46.500000,43.750000,43.000000,286.250000,279.500000,272.750000,137.750000,135.500000,132.500000
50%,60.500000,194.500000,95.000000,528.000000,263.500000,0.000000,0.000000,0.000000,193.000000,192.000000,192.000000,95.000000,94.500000,93.500000,527.500000,522.000000,513.000000,259.500000,258.500000,256.500000
75%,90.250000,217.000000,146.250000,776.000000,372.000000,1.000000,1.000000,1.000000,217.000000,217.000000,217.000000,146.250000,144.500000,143.250000,769.500000,761.250000,759.250000,372.000000,371.000000,370.250000
max,120.000000,252.000000,200.000000,999.000000,496.000000,1.000000,1.000000,1.000000,252.000000,252.000000,252.000000,200.000000,200.000000,200.000000,999.000000,999.000000,999.000000,496.000000,496.000000,496.000000


In [26]:
spec_cols = ['Specialty_Hematologist', 'Specialty_Neurologist', 'Specialty_Oncologist']
df_t_n = df_t[df_t['Specialty']=='Neurologist'].drop(spec_cols, axis=1)
df_t_o = df_t[df_t['Specialty']=='Oncologist'].drop(spec_cols, axis=1)
df_t_h = df_t[df_t['Specialty']=='Hematologist'].drop(spec_cols, axis=1)

In [27]:
df_t_n

,Specialty,date,rx_count,Email,Phone,Digital,rx_count_lag1,rx_count_lag2,rx_count_lag3,Email_lag1,Email_lag2,Email_lag3,Phone_lag1,Phone_lag2,Phone_lag3,Digital_lag1,Digital_lag2,Digital_lag3
0,Neurologist,1,199,185,523,44,0,0,0,0,0,0,0,0,0,0,0,0
1,Neurologist,2,167,3,910,446,199,0,0,185,0,0,523,0,0,44,0,0
2,Neurologist,3,212,81,864,147,167,199,0,3,185,0,910,523,0,446,44,0
3,Neurologist,4,214,141,998,131,212,167,199,81,3,185,864,910,523,147,446,44
4,Neurologist,5,214,38,888,427,214,212,167,141,81,3,998,864,910,131,147,446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Neurologist,116,193,15,561,209,138,146,204,106,61,31,7,208,687,238,130,164
116,Neurologist,117,218,113,883,282,193,138,146,15,106,61,561,7,208,209,238,130
117,Neurologist,118,141,52,140,422,218,193,138,113,15,106,883,561,7,282,209,238
118,Neurologist,119,207,181,831,50,141,218,193,52,113,15,140,883,561,422,282,209


In [28]:
fig = px.scatter(df_t_n, x="Email", y='rx_count')
fig.show()

# Model Fitting

In [29]:
modeling = mmm_modeling.MMMModeling()

In [30]:
# modeling
channels = ['Email', 'Phone', 'Digital']
lag_dv = [x for x in df_t.columns if 'rx_count_lag' in x]
lag_channels = [x for x in df_t.columns if ('lag' in x) & ('rx_count' not in x)]
X = df_t[channels + lag_channels + lag_dv]
y = df_t['rx_count']
model_n = modeling.rf_regressor(df_t_n, X.columns.tolist(), 'rx_count', 'date')
model_o = modeling.rf_regressor(df_t_o, X.columns.tolist(), 'rx_count', 'date')
model_h = modeling.rf_regressor(df_t_h, X.columns.tolist(), 'rx_count', 'date')

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_modeling.py:55: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_modeling.py:56: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_modeling.py:55: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [31]:
# performance
model_n['performance']

{'full': {'r2': 0.9791210452185023,
  'rmse': 4.521884931456499,
  'mape': 0.017216550392528722},
 'train': {'r2': 0.9805150591525365,
  'rmse': 4.280345342376009,
  'mape': 0.017520282446003545},
 'test': {'r2': 0.8883172312223859,
  'rmse': 10.885239432368953,
  'mape': 0.04895993433238458}}

In [32]:
model_o['performance']

{'full': {'r2': 0.9904196223101025,
  'rmse': 1.8282017576478444,
  'mape': 0.008215230881101201},
 'train': {'r2': 0.9843227712386181,
  'rmse': 2.4000099826181285,
  'mape': 0.010871578219974242},
 'test': {'r2': 0.9631276379506187,
  'rmse': 3.1043417176593175,
  'mape': 0.01604509302280135}}

In [33]:
model_h['performance']

{'full': {'r2': 0.9898856978171856,
  'rmse': 3.651853410712611,
  'mape': 0.014523959717282956},
 'train': {'r2': 0.9880652671884341,
  'rmse': 4.074547648717176,
  'mape': 0.016977933981767057},
 'test': {'r2': 0.943162816031217,
  'rmse': 7.633010546304778,
  'mape': 0.027741570110858548}}

In [34]:
# importance
model_n['importance']

,feature,importance,std
1,Phone,0.767017,0.066403
0,Email,0.164862,0.070229
2,Digital,0.017204,0.012045
10,Digital_lag2,0.006990,0.019049
11,Digital_lag3,0.006962,0.013598
7,Phone_lag2,0.006580,0.020689
9,Digital_lag1,0.005614,0.012006
8,Phone_lag3,0.005550,0.011467
13,rx_count_lag2,0.004912,0.012196
6,Phone_lag1,0.003518,0.006309


In [35]:
model_o['importance']

,feature,importance,std
2,Digital,0.569057,0.104926
1,Phone,0.359751,0.114935
11,Digital_lag3,0.014284,0.026574
5,Email_lag3,0.007547,0.010230
8,Phone_lag3,0.007427,0.016641
14,rx_count_lag3,0.006822,0.015454
0,Email,0.005676,0.016985
12,rx_count_lag1,0.005152,0.012766
6,Phone_lag1,0.004934,0.011182
7,Phone_lag2,0.004719,0.013207


In [36]:
model_h['importance']

,feature,importance,std
2,Digital,0.740802,0.084687
0,Email,0.098536,0.046363
1,Phone,0.087561,0.043060
13,rx_count_lag2,0.016069,0.032335
11,Digital_lag3,0.013639,0.031894
10,Digital_lag2,0.009965,0.024421
4,Email_lag2,0.007889,0.021630
7,Phone_lag2,0.004323,0.012488
9,Digital_lag1,0.004107,0.007373
5,Email_lag3,0.003846,0.007739


# Response Curves - Neurologist

In [37]:
response_curves = mmm_response_curves.MMMResponseCurves()

In [38]:
channels = ['Email', 'Phone', 'Digital']
lag_dv = [x for x in df_t_n.columns if 'rx_count_lag' in x]
lag_channels = [x for x in df_t_n.columns if ('lag' in x) & ('rx_count' not in x)]
X = df_t_n[channels + lag_channels + lag_dv]
#X[channels + lag_channels] = 0

In [39]:
df_t_n.describe()

,date,rx_count,Email,Phone,Digital,rx_count_lag1,rx_count_lag2,rx_count_lag3,Email_lag1,Email_lag2,Email_lag3,Phone_lag1,Phone_lag2,Phone_lag3,Digital_lag1,Digital_lag2,Digital_lag3
count,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000
mean,60.500000,191.475000,93.758333,553.983333,267.875000,189.658333,187.933333,186.758333,92.991667,91.483333,91.050000,546.008333,539.083333,537.916667,265.225000,264.808333,261.291667
std,34.785054,31.425505,60.272908,269.328046,133.429902,35.866582,39.788883,43.128301,60.877546,60.920941,61.387302,271.454486,274.703311,276.702479,135.566751,136.308615,137.656093
min,1.000000,50.000000,0.000000,7.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.750000,177.250000,43.000000,372.500000,165.500000,175.000000,174.500000,174.500000,41.750000,37.750000,36.500000,358.000000,345.500000,345.500000,159.750000,159.750000,145.500000
50%,60.500000,203.000000,83.500000,541.000000,283.500000,202.500000,201.500000,201.500000,83.000000,82.500000,82.500000,538.000000,532.500000,532.500000,281.500000,281.500000,275.000000
75%,90.250000,214.000000,148.500000,791.500000,380.250000,214.000000,214.000000,214.000000,148.500000,142.750000,142.750000,785.750000,782.500000,782.500000,380.250000,380.250000,377.750000
max,120.000000,221.000000,200.000000,998.000000,493.000000,221.000000,221.000000,221.000000,200.000000,200.000000,200.000000,998.000000,998.000000,998.000000,493.000000,493.000000,493.000000


In [40]:
# overall response curves
channel1_n = response_curves.responses(model_n['full_model'], X, 'Email', 200, 1)
channel2_n = response_curves.responses(model_n['full_model'], X, 'Phone', 1000, 10)
channel3_n = response_curves.responses(model_n['full_model'], X, 'Digital', 500, 5)

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:22: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_cur

In [41]:
response_curves.plot(channel1_n['resp_df'], 'touches', ['Email', 'Email_hill_estimate'])

In [42]:
response_curves.plot(channel2_n['resp_df'], 'touches', ['Phone', 'Phone_hill_estimate'])

In [43]:
response_curves.plot(channel3_n['resp_df'], 'touches', ['Digital', 'Digital_hill_estimate'])

In [81]:
channel3['resp_df'].to_csv('D:/Users/hartsingh/Documents/Projects/Misc output/digital_n.csv', index=False)

# Response Curves - Oncologist

In [44]:
channels = ['Email', 'Phone', 'Digital']
lag_dv = [x for x in df_t_o.columns if 'rx_count_lag' in x]
lag_channels = [x for x in df_t_o.columns if ('lag' in x) & ('rx_count' not in x)]
X = df_t_o[channels + lag_channels + lag_dv]
#X[channels + lag_channels] = 0

In [45]:
df_t_o.describe()

,date,rx_count,Email,Phone,Digital,rx_count_lag1,rx_count_lag2,rx_count_lag3,Email_lag1,Email_lag2,Email_lag3,Phone_lag1,Phone_lag2,Phone_lag3,Digital_lag1,Digital_lag2,Digital_lag3
count,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000
mean,60.500000,145.858333,97.616667,505.566667,259.591667,144.850000,143.566667,142.291667,97.100000,97.008333,97.000000,503.041667,497.958333,493.358333,258.758333,257.291667,254.850000
std,34.785054,18.756420,57.008356,280.657354,137.129424,22.898993,26.425648,29.481057,57.611623,57.758334,57.772373,283.840066,287.349037,290.873353,138.405323,140.210430,142.121539
min,1.000000,83.000000,1.000000,7.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.750000,137.000000,52.500000,278.250000,143.000000,137.000000,134.000000,133.500000,50.750000,50.750000,50.750000,270.500000,257.250000,236.500000,143.000000,142.750000,141.250000
50%,60.500000,154.500000,94.500000,534.000000,254.500000,154.500000,154.500000,154.500000,94.500000,94.500000,94.500000,534.000000,522.500000,504.500000,254.500000,254.500000,251.000000
75%,90.250000,160.000000,146.000000,729.250000,363.250000,160.000000,160.000000,160.000000,146.000000,146.000000,146.000000,729.250000,729.250000,729.250000,363.250000,363.250000,363.250000
max,120.000000,163.000000,199.000000,999.000000,496.000000,163.000000,163.000000,163.000000,199.000000,199.000000,199.000000,999.000000,999.000000,999.000000,496.000000,496.000000,496.000000


In [46]:
# overall response curves
channel1_o = response_curves.responses(model_o['full_model'], X, 'Email', 200, 1)
channel2_o = response_curves.responses(model_o['full_model'], X, 'Phone', 1000, 10)
channel3_o = response_curves.responses(model_o['full_model'], X, 'Digital', 500, 5)

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:22: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_cur

In [47]:
response_curves.plot(channel1_o['resp_df'], 'touches', ['Email', 'Email_hill_estimate'])

In [48]:
response_curves.plot(channel2_o['resp_df'], 'touches', ['Phone', 'Phone_hill_estimate'])

In [49]:
response_curves.plot(channel3_o['resp_df'], 'touches', ['Digital', 'Digital_hill_estimate'])

In [65]:
channel3['resp_df'].to_csv('D:/Users/hartsingh/Documents/Projects/Misc output/digital_o.csv', index=False)

# Response Curves - Hematologist

In [50]:
channels = ['Email', 'Phone', 'Digital']
lag_dv = [x for x in df_t_h.columns if 'rx_count_lag' in x]
lag_channels = [x for x in df_t_h.columns if ('lag' in x) & ('rx_count' not in x)]
X = df_t_o[channels + lag_channels + lag_dv]
#X[channels + lag_channels] = 0

In [51]:
df_t_h.describe()

,date,rx_count,Email,Phone,Digital,rx_count_lag1,rx_count_lag2,rx_count_lag3,Email_lag1,Email_lag2,Email_lag3,Phone_lag1,Phone_lag2,Phone_lag3,Digital_lag1,Digital_lag2,Digital_lag3
count,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.00000,120.000000
mean,60.500000,216.366667,96.000000,516.366667,241.900000,214.700000,212.608333,211.783333,95.858333,94.408333,92.891667,514.316667,507.583333,499.800000,239.091667,235.67500,235.658333
std,34.785054,36.463850,57.120395,302.786219,147.567925,41.448277,45.714734,48.585398,57.338637,57.545670,57.616057,305.453128,307.821338,308.755474,148.943168,149.69039,149.716735
min,1.000000,58.000000,1.000000,23.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,30.750000,209.000000,45.750000,258.000000,119.500000,209.000000,208.750000,208.750000,45.750000,41.750000,40.500000,258.000000,251.000000,237.250000,115.500000,106.50000,106.500000
50%,60.500000,225.500000,101.000000,504.500000,253.000000,225.500000,224.000000,224.000000,101.000000,100.500000,99.000000,504.500000,494.500000,487.000000,250.000000,248.00000,248.000000
75%,90.250000,242.000000,144.750000,798.250000,370.250000,242.000000,241.250000,241.250000,144.750000,143.250000,143.000000,798.250000,795.750000,790.500000,370.250000,368.50000,368.500000
max,120.000000,252.000000,198.000000,995.000000,493.000000,252.000000,252.000000,252.000000,198.000000,198.000000,198.000000,995.000000,995.000000,995.000000,493.000000,493.00000,493.000000


In [52]:
# overall response curves
channel1_h = response_curves.responses(model_h['full_model'], X, 'Email', 200, 1)
channel2_h = response_curves.responses(model_h['full_model'], X, 'Phone', 1000, 10)
channel3_h = response_curves.responses(model_h['full_model'], X, 'Digital', 500, 5)

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:22: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_cur

In [53]:
response_curves.plot(channel1_h['resp_df'], 'touches', ['Email', 'Email_hill_estimate'])

In [54]:
response_curves.plot(channel2_h['resp_df'], 'touches', ['Phone', 'Phone_hill_estimate'])

In [55]:
response_curves.plot(channel3_h['resp_df'], 'touches', ['Digital', 'Digital_hill_estimate'])

In [151]:
channel3['optimal_hill']

array([130.93790319,  29.24294816,   1.74773258])

In [90]:
channel3['resp_df'].to_csv('D:/Users/hartsingh/Documents/Projects/Misc output/digital_h.csv', index=False)

# Neurologist Optimization - Hyperopt 

In [60]:
neuro_hill = pd.concat([pd.DataFrame(channel1_n['optimal_hill']).T,
                          pd.DataFrame(channel2_n['optimal_hill']).T,
                          pd.DataFrame(channel3_n['optimal_hill']).T])
neuro_hill.columns = ['beta', 'gamma', 'alpha']
neuro_hill['feature'] = ['Email', 'Phone', 'Digital']
neuro_hill

,beta,gamma,alpha,feature
0,46.633193,11.928508,2.026613,Email
0,79.256993,309.583473,4.311753,Phone
0,6.444855,118.903393,2.624504,Digital


In [62]:
df_t_n.describe()

,date,rx_count,Email,Phone,Digital,rx_count_lag1,rx_count_lag2,rx_count_lag3,Email_lag1,Email_lag2,Email_lag3,Phone_lag1,Phone_lag2,Phone_lag3,Digital_lag1,Digital_lag2,Digital_lag3
count,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000
mean,60.500000,191.475000,93.758333,553.983333,267.875000,189.658333,187.933333,186.758333,92.991667,91.483333,91.050000,546.008333,539.083333,537.916667,265.225000,264.808333,261.291667
std,34.785054,31.425505,60.272908,269.328046,133.429902,35.866582,39.788883,43.128301,60.877546,60.920941,61.387302,271.454486,274.703311,276.702479,135.566751,136.308615,137.656093
min,1.000000,50.000000,0.000000,7.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.750000,177.250000,43.000000,372.500000,165.500000,175.000000,174.500000,174.500000,41.750000,37.750000,36.500000,358.000000,345.500000,345.500000,159.750000,159.750000,145.500000
50%,60.500000,203.000000,83.500000,541.000000,283.500000,202.500000,201.500000,201.500000,83.000000,82.500000,82.500000,538.000000,532.500000,532.500000,281.500000,281.500000,275.000000
75%,90.250000,214.000000,148.500000,791.500000,380.250000,214.000000,214.000000,214.000000,148.500000,142.750000,142.750000,785.750000,782.500000,782.500000,380.250000,380.250000,377.750000
max,120.000000,221.000000,200.000000,998.000000,493.000000,221.000000,221.000000,221.000000,200.000000,200.000000,200.000000,998.000000,998.000000,998.000000,493.000000,493.000000,493.000000


In [63]:
# current impact
optimization = mmm_optimization.MMMOptimization(budget=750, params=neuro_hill)
imp = optimization.calc_impact([94, 554, 268])
imp

{'impact': 124.99056931920633, 'total spend': 916}

In [125]:
# optimized spend using hill - no interactions
optimization = mmm_optimization.MMMOptimization(budget=916, params=neuro_hill)
channels = neuro_hill['feature'].tolist()
output = optimization.optimize_hyperopt_hill(channels, 2000)
output

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:39<00:00, 50.21trial/s, best loss: -125.53066810514927]


{'mix': {'Digital': 109, 'Email': 71, 'Phone': 716},
 'total_spend': 896,
 'trials': [{'loss': -69.45087711833435, 'status': 'ok'},
  {'loss': 99999892.9695586, 'status': 'ok'},
  {'loss': 99999903.20732604, 'status': 'ok'},
  {'loss': 99999897.355319, 'status': 'ok'},
  {'loss': 99999875.84125774, 'status': 'ok'},
  {'loss': 99999872.74446605, 'status': 'ok'},
  {'loss': 99999945.98180272, 'status': 'ok'},
  {'loss': 99999922.20143571, 'status': 'ok'},
  {'loss': 99999872.7862767, 'status': 'ok'},
  {'loss': 99999869.84834781, 'status': 'ok'},
  {'loss': 99999903.51472604, 'status': 'ok'},
  {'loss': -51.05912090328378, 'status': 'ok'},
  {'loss': 99999868.57149981, 'status': 'ok'},
  {'loss': 99999879.40974009, 'status': 'ok'},
  {'loss': -52.953479447788936, 'status': 'ok'},
  {'loss': 99999874.88649186, 'status': 'ok'},
  {'loss': 99999943.53318813, 'status': 'ok'},
  {'loss': 99999886.3019586, 'status': 'ok'},
  {'loss': 99999871.29883143, 'status': 'ok'},
  {'loss': 99999871.0216

In [126]:
# optimized spend using model - includes interactions
optimization = mmm_optimization.MMMOptimization(budget=916)
lag_dv = [x for x in df_t_n.columns if 'rx_count_lag' in x]
lag_channels = [x for x in df_t_n.columns if ('lag' in x) & ('rx_count' not in x)]
X = df_t_n[channels + lag_channels + lag_dv]
output = optimization.optimize_predict(X, channels, 2000, model_n['full_model'], 1)
output

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:14<00:00, 26.79trial/s, best loss: -115.33233333333334]


{'mix': {'Digital': 141, 'Email': 105, 'Phone': 614},
 'total_spend': 860,
 'baseline_impact': 95.91383333333333,
 'trials': [{'loss': 99999885.56125, 'status': 'ok'},
  {'loss': 99999958.86708333, 'status': 'ok'},
  {'loss': 99999890.81458333, 'status': 'ok'},
  {'loss': -39.85841666666667, 'status': 'ok'},
  {'loss': 99999964.33708334, 'status': 'ok'},
  {'loss': -73.36733333333333, 'status': 'ok'},
  {'loss': 99999936.43583333, 'status': 'ok'},
  {'loss': -30.460083333333337, 'status': 'ok'},
  {'loss': 99999889.55241667, 'status': 'ok'},
  {'loss': 99999965.07875, 'status': 'ok'},
  {'loss': 99999906.06475, 'status': 'ok'},
  {'loss': 99999915.706, 'status': 'ok'},
  {'loss': -74.47175, 'status': 'ok'},
  {'loss': 99999889.17875, 'status': 'ok'},
  {'loss': 99999945.39466667, 'status': 'ok'},
  {'loss': 99999888.051, 'status': 'ok'},
  {'loss': 99999929.965, 'status': 'ok'},
  {'loss': 99999898.54125, 'status': 'ok'},
  {'loss': -32.688750000000006, 'status': 'ok'},
  {'loss': 9999

In [113]:
channels

['Email', 'Phone', 'Digital']

In [118]:
optimization.calc_impact_model(X, channels, model_n['full_model'], [94, 554, 268])

{'impact': 113.43233333333335, 'total spend': 916}

In [127]:
optimization.calc_impact_model(X, channels, model_n['full_model'], [71, 716, 109])

{'impact': 111.90316666666668, 'total spend': 896}

In [109]:
channel3_n['resp_og']

,touches,Digital
0,0,186.527167
1,5,186.747833
2,10,186.927167
3,15,187.127417
4,20,187.127417
...,...,...
96,480,191.813917
97,485,191.770917
98,490,191.480167
99,495,191.480167


In [110]:
channel3_n['resp_df']

,touches,Digital,Digital_hill_estimate,touches_scaled,Digital_hill_estimate_minmax
0,0.0,0.000000,0.000000,0.00,0.000000
1,5.0,0.220667,0.001575,0.01,0.000250
2,10.0,0.400000,0.009699,0.02,0.001540
3,15.0,0.600250,0.028030,0.03,0.004449
4,20.0,0.600250,0.059347,0.04,0.009421
...,...,...,...,...,...
96,480.0,5.286750,6.283556,0.96,0.997457
97,485.0,5.243750,6.287778,0.97,0.998127
98,490.0,4.953000,6.291851,0.98,0.998774
99,495.0,4.953000,6.295781,0.99,0.999398


# Digital Optimization - Hill

In [56]:
df_plot = pd.DataFrame({'touches': channel3_n['resp_df']['touches'],
                        'digital_n': channel3_n['resp_df']['Digital_hill_estimate'],
                        'digital_o': channel3_o['resp_df']['Digital_hill_estimate'],
                        'digital_h': channel3_h['resp_df']['Digital_hill_estimate']})
response_curves.plot(df_plot, 'touches', ['digital_n', 'digital_o', 'digital_h'])

In [178]:
digital_hill = pd.concat([pd.DataFrame(channel3_n['optimal_hill']).T,
                          pd.DataFrame(channel3_o['optimal_hill']).T,
                          pd.DataFrame(channel3_h['optimal_hill']).T])
digital_hill['segment'] = ['N', 'O', 'H']
digital_hill.columns = ['beta', 'gamma', 'alpha', 'feature']
digital_hill

,beta,gamma,alpha,feature
0,7.921299,91.412755,1.870447,N
0,42.530765,125.371314,7.450854,O
0,130.937903,29.242948,1.747733,H


In [191]:
optimization = mmm_optimization.MMMOptimization(budget=769, params=digital_hill)
start_vals = [100, 100, 100]
output = optimization.optimize_hill(start_vals)
output

-1*((7.921298838162023*(1/(1+(n[0]/91.41275456254392)**(-1.8704470641254065))))+(42.53076461169334*(1/(1+(n[1]/125.3713138983588)**(-7.450853709127819))))+(130.93790319195642*(1/(1+(n[2]/29.242948164898635)**(-1.7477325774874637)))))
[207.46509906 239.6287677  321.90613325]
     fun: -177.69590060953857
     jac: array([-0.01042938, -0.01042938, -0.01042938])
 message: 'Optimization terminated successfully'
    nfev: 117
     nit: 29
    njev: 29
  status: 0
 success: True
       x: array([207.46509906, 239.6287677 , 321.90613325])


In [199]:
# optimal 1 - use all budget
imp = optimization.calc_impact([207.46509906, 239.6287677 , 321.90613325])
imp

{'impact': 177.69590060964285, 'total spend': 769.0000000099999}

In [204]:
# optimal 2 - saver
imp = optimization.calc_impact([200, 200, 200])
imp

{'impact': 174.2370212725674, 'total spend': 600}

In [200]:
# current
imp = optimization.calc_impact([267.875000, 259.591667, 241.900000])
imp

{'impact': 177.08637224154015, 'total spend': 769.3666669999999}

# Bootstrap Prediction Intervals

In [219]:
channels = ['Email', 'Phone', 'Digital']
lag_dv = [x for x in df_t.columns if 'rx_count_lag' in x]
lag_channels = [x for x in df_t.columns if ('lag' in x) & ('rx_count' not in x)]
X = df_t_n[channels + lag_channels + lag_dv]
y = df_t_n['rx_count']
# Generate bootstrapped predictions for each test observation
n_bootstraps = 1000
bootstrapped_preds = np.zeros((X.shape[0], n_bootstraps))
# this will create a 120 x 1000 array where each row is the 1000 predictions of y at that row
for i in range(n_bootstraps):
    bootstrapped_samples = np.random.choice(X.shape[0], size=X.shape[0], replace=True)
    X_bootstrapped = X.iloc[bootstrapped_samples]
    y_bootstrapped = y[bootstrapped_samples]
    rf_bootstrapped = RandomForestRegressor()
    rf_bootstrapped.fit(X_bootstrapped, y_bootstrapped)
    bootstrapped_preds[:, i] = rf_bootstrapped.predict(X)
# Calculate the prediction interval for each test observation
lower_percentile = 2.5
upper_percentile = 97.5
lower_bounds = np.percentile(bootstrapped_preds, lower_percentile, axis=1)
upper_bounds = np.percentile(bootstrapped_preds, upper_percentile, axis=1)
prediction_intervals = np.column_stack((lower_bounds, upper_bounds))

# this will give us prediction intervals for each row but when generating response curves we average predictions across all rows
# the error needs to be propogated in this average to get the final prediction interval

In [222]:
prediction_intervals

array([[197.82525, 205.90025],
       [152.84325, 199.4445 ],
       [207.2895 , 212.5805 ],
       [207.01   , 213.98   ],
       [205.5385 , 214.85125],
       [217.17975, 220.9505 ],
       [194.62675, 204.91025],
       [142.46725, 182.13375],
       [198.31525, 209.     ],
       [187.018  , 202.0005 ],
       [185.70725, 204.86075],
       [197.87875, 205.242  ],
       [211.98725, 217.16   ],
       [212.85725, 216.9005 ],
       [174.64875, 192.77075],
       [183.28975, 199.474  ],
       [185.6175 , 203.21075],
       [187.02925, 202.64675],
       [195.18   , 203.76025],
       [151.38375, 175.58   ],
       [201.17975, 209.162  ],
       [167.3395 , 192.1715 ],
       [211.6575 , 217.20075],
       [191.8185 , 204.52025],
       [199.50925, 209.94   ],
       [194.44775, 204.292  ],
       [206.08825, 211.24075],
       [198.176  , 209.37025],
       [144.71975, 179.264  ],
       [183.689  , 199.78125],
       [130.759  , 152.0515 ],
       [147.09675, 178.87025],
       [

In [226]:
bootstrapped_preds

array([[200.87, 201.98, 199.31, ..., 198.84, 201.05, 201.  ],
       [173.68, 198.16, 167.32, ..., 179.1 , 174.05, 173.91],
       [211.42, 212.  , 210.52, ..., 207.99, 211.92, 211.01],
       ...,
       [121.04, 140.23, 142.06, ..., 138.47, 128.37, 110.77],
       [206.89, 205.74, 208.1 , ..., 205.64, 208.41, 206.91],
       [218.01, 217.44, 217.83, ..., 210.49, 216.89, 217.66]])

In [225]:
bootstrapped_preds.shape

(120, 1000)

In [232]:
bootstrapped_preds[:,0]

array([200.87, 173.68, 211.42, 212.5 , 212.  , 220.47, 203.83, 166.11,
       207.13, 199.76, 201.24, 201.15, 216.47, 214.82, 176.44, 194.56,
       195.75, 198.26, 202.52, 174.35, 203.82, 171.06, 216.06, 204.  ,
       207.94, 198.25, 209.93, 207.95, 172.31, 194.92, 138.4 , 172.52,
       211.86, 218.87, 216.2 , 206.68, 199.96, 210.86, 217.16, 179.58,
       213.  , 172.42, 215.27, 124.21, 137.69, 212.03, 140.41, 214.9 ,
       210.91,  60.39, 133.29, 212.54, 161.83, 208.13, 216.86, 206.65,
       215.58, 150.66, 144.98, 138.4 , 201.41, 211.77, 219.91, 211.21,
       205.48, 216.14, 217.22, 210.91, 218.84, 209.05, 115.22, 143.22,
       216.7 , 211.7 , 218.4 , 219.51, 135.63, 207.71, 180.76, 216.05,
       147.83, 197.67, 151.99, 216.84, 107.8 , 202.13, 212.36, 197.08,
       202.33, 217.01, 194.36, 211.3 , 200.77, 198.52, 213.94, 205.25,
       207.58, 217.16, 216.8 , 107.98, 190.01, 132.89, 209.3 , 184.25,
       210.17, 183.11, 177.21, 203.78, 195.15, 140.11, 191.71, 171.53,
      